In [2]:
import numpy as np
from joblib import Parallel, delayed
from time import time
from tqdm import tqdm
from sklearn.neighbors import KDTree

p1=np.array([[0,0,0],[0,0,0]])
p2=np.array([[10,10,0],[10,10,0]])
p3=np.array([[5,7,0],[5,7,0]])
pt=np.array([[1,2,5],[1,2,5]])

p1 = np.repeat(p1, 200_000, axis=0)
p2 = np.repeat(p2, 200_000, axis=0)
p3 = np.repeat(p3, 200_000, axis=0)
pt = np.repeat(pt, 200_000, axis=0)

pc_surf = np.stack((p1, p2, p3), axis=2)
block_size = 1_000
ncpu = 2

def return_batch_dist(a,b):
    return np.diag(np.dot(a,b.T))

def compute_plane_dist(pc_surf, pc_pt, block_size, ncpu):
    n_dims = np.shape(pc_surf)[1]
    if n_dims <3:
        pc_surf = np.concatenate((pc_surf, np.zeros((np.shape(pc_surf)[0], 1))), axis=1)
        pc_pt = np.concatenate((pc_pt, np.zeros((np.shape(pc_pt)[0], 1))), axis=1)

    p1 = pc_surf[:,:,0] # n_samples, x/y/z, p1/2/3
    p2 = pc_surf[:,:,1]
    p3 = pc_surf[:,:,2]

    v21 = p2-p1
    v31 = p3-p1
    vt1 = pt-p1
    n = np.cross(v21,v31)
    n = n/np.linalg.norm(n, axis=1).reshape(-1,1)

    dd = Parallel(n_jobs=ncpu)\
         (delayed(return_batch_dist)(vt1[pos:pos + block_size, :], n[pos:pos + block_size, :]) \
         for pos in tqdm(range(0, vt1.shape[0], block_size)))
    sdf = np.concatenate(dd, axis=0)
    return sdf

def get_sdf(surf_points, pc_pt, block_size, ncpu):
    kdt = KDTree(surf_points, leaf_size=3)
    _, ix = kdt.query(pc_pt, k=3)
    pc_surf = surf_points[ix]
    sdf = compute_plane_dist(pc_surf, pc_pt, block_size, ncpu)
    return sdf


sdf = compute_plane_dist(pc_surf, pt, block_size, ncpu)


100%|██████████| 400/400 [00:02<00:00, 146.80it/s]


In [3]:
sdf

array([5., 5., 5., ..., 5., 5., 5.])

In [4]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()


In [5]:
# import condacolab
# condacolab.check()


In [6]:
# !conda install -c conda-forge plaid

In [7]:
#  !conda install conda-forge::muscat

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# fname = '/content/drive/MyDrive/mesh_000000000.cgns'

In [10]:
from typing import Optional, Union
import os
from tqdm import tqdm
import numpy as np

In [11]:


# from Muscat.Types import MuscatFloat, MuscatIndex
# from Muscat.Bridges.CGNSBridge import CGNSToMesh
# from Muscat.Containers.Mesh import Mesh

# def ReadCGNS(fileName, time: Optional[MuscatFloat] = None, baseNumberOrName: Union[MuscatIndex, str] = 0, zoneNumberOrName: Union[MuscatIndex, str] = 0) -> Mesh:
#     """Function API for reading a CGNS file

#     Parameters
#     ----------
#     fileName : str
#         name of the file to be read
#     time : float, optional
#         not coded yet, by default None
#     baseNumberOrName : int or str, optional
#         name of the base to use, by default 0 (first)
#     zoneNumberOrName : int or str, optional
#         name of the zone to be read, by default 0 (first)

#     Returns
#     -------
#     Mesh
#         output unstructured mesh object containing reading result
#     """
#     reader = CGNSReader()
#     reader.SetFileName(fileName)
#     reader.baseNumberOrName: Union[MuscatIndex, str] = baseNumberOrName
#     reader.zoneNumberOrName: Union[MuscatIndex, str] = zoneNumberOrName
#     reader.SetTimeToRead(time)
#     res = reader.Read(fileName=fileName)
#     return res


# class CGNSReader:
#     """CGNS Reader class"""

#     def __init__(self) -> None:
#         super().__init__()
#         self.fileName: str = None
#         self.fieldName = None
#         self.baseNumberOrName = None
#         self.zoneNumberOrName = None
#         self.timeToRead: MuscatFloat = -1.0

#         self.encoding = None
#         self.canHandleTemporal = False

#     def SetFileName(self, fileName: str) -> None:
#         """Function to set fileName to read

#         Parameters
#         ----------
#         fileName : str
#             name of the file to be read
#         """
#         self.fileName = fileName
#         if fileName is None:
#             self.__path = None
#         else:
#             self.filePath = os.path.abspath(os.path.dirname(fileName)) + os.sep

#     def SetTimeToRead(self, time: Optional[MuscatFloat] = None) -> None:
#         """Function to set time value to read

#         Parameters
#         ----------
#         time : float, optional
#             not coded yet, by default None
#         """
#         if time is None:
#             self.timeToRead = 0.0
#         else:  # pragma: no cover
#             raise NotImplementedError("not coded yet")
#             self.timeToRead = time

#     def Read(self, fileName: Optional[str] = None, time: Optional[MuscatFloat] = None) -> Mesh:
#         """Function that performs the reading of a CGNS result file

#         Parameters
#         ----------
#         fileName : str
#             name of the file to be read
#         time : float, optional
#             not coded yet, by default None
#         baseNumberOrName : int or str, optional
#             name of the base to use, by default 0 (first)
#         zoneNumberOrName : int or str, optional
#             name of the zone to be read, by default 0 (first)

#         Returns
#         -------
#         Mesh
#             output unstructured mesh object containing reading result
#         """
#         if fileName is not None:
#             self.SetFileName(fileName)

#         self.SetTimeToRead(time)

#         from h5py import File

#         h5file = File(self.fileName, "r")

#         def ConvertData(node):
#             res = [node.attrs["name"].decode("utf-8"), None, [], node.attrs["label"].decode("utf-8")]
#             if " data" in node:
#                 dataitem = node[" data"]
#                 res[1] = np.copy(dataitem[()].transpose(), order="F")
#                 if node.attrs["type"] == b"C1":
#                     res[1] = np.vectorize(chr)(res[1]).astype(np.dtype("c"))

#             names = [x for x in node.keys() if x[0] != " "]
#             for name in names:
#                 child = ConvertData(node[name])
#                 if child is not None:
#                     res[2].append(child)
#             return res

#         node = ConvertData(h5file)
#         node[0] = "CGNSTree"
#         node[3] = "CGNSTree_t"
#         self.CGNSTree = node

#         res = CGNSToMesh(node)
#         res.PrepareForOutput()
#         return res

In [12]:
# mm = ReadCGNS(fname)

In [13]:
# dir(mm)

In [14]:
# np.shape(mm.nodes)

In [15]:
# mm.nodeFields

In [16]:
# mm.nodeFields.keys()

In [17]:
# np.shape(mm.nodeFields['Density'])

In [18]:
!pip install pyvista
!pip install UQpy


In [19]:
import pyvista as pv
from UQpy.dimension_reduction.pod.DirectPOD import DirectPOD


In [20]:
# mesh_out = pv.PolyData(mm.nodes)
# for k, f in mm.nodeFields.items():
#     mesh_out[k] = f


In [21]:
# mesh_out.save('/content/drive/MyDrive/mesh_000000000.vtk')


In [22]:
# folder = '/content/drive/MyDrive/samples_rotor37/'
# for ca in tqdm(os.listdir(folder)):
#     try:
#         int_folder = folder + ca + '/meshes/'
#         ff = os.listdir(int_folder)[0]
#         fname = int_folder + ff
#         fname_out = fname.replace('.cgns', '.vtk')
#         mm = ReadCGNS(fname)
#         mesh_out = pv.PolyData(mm.nodes)
#         for k, f in mm.nodeFields.items():
#             mesh_out[k] = f
#         mesh_out.save(fname_out)
#     except:
#         print(ca)


In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import pandas as pd


In [24]:
# folder = '/content/drive/MyDrive/samples_rotor37/'
# for ca in tqdm(os.listdir(folder)[:1000]):
#     try:
#         int_folder = folder + ca + '/meshes/'
#         ff = [f for f in os.listdir(int_folder) if '.vtk' in f][0]
#         fname = int_folder + ff
#         gg = pv.read(fname)
#         x = gg.points.reshape(1,-1)
#         y = gg.point_data['Pressure'].reshape(1,-1)

#         tmp = pd.read_csv(folder + ca + '/scalars.csv').values

#         if 'x_all' in locals():
#             x_scalar = np.concatenate((x_scalar, tmp), axis=0)
#             x_all = np.concatenate((x_all, x), axis=0)
#             y_all = np.concatenate((y_all, y), axis=0)
#         else:
#             x_scalar = tmp
#             x_all = x
#             y_all = y
#     except:
#         print(ca)

In [25]:
from sklearn.linear_model import Ridge, BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.multioutput import MultiOutputRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern, ConstantKernel
from sklearn.metrics import mean_squared_error, r2_score

In [26]:
# np.save('/content/drive/MyDrive/x_all.npy', x_all)
# np.save('/content/drive/MyDrive/y_all.npy', y_all)
# np.save('/content/drive/MyDrive/x_scalar.npy', x_scalar)

x_all = np.load('/content/drive/MyDrive/x_all.npy')
y_all = np.load('/content/drive/MyDrive/y_all.npy')
x_scalar = np.load('/content/drive/MyDrive/x_scalar.npy')


In [27]:
ix_train = np.arange(0, 600, 1)
ix_test = np.delete(range(len(x_all)), ix_train)

x_train = x_all[ix_train, :]
x_test = x_all[ix_test, :]
x_scalar_train = x_scalar[ix_train, -2:]

y_train = y_all[ix_train, :]
y_test = y_all[ix_test, :]
x_scalar_test = x_scalar[ix_test, -2:]


In [28]:
np.shape(y_train)

(600, 29773)

In [29]:
np.min(y_train, axis=0)

array([115823.43381169, 115812.91447501, 115803.36256068, ...,
        78041.59912412,  77992.64621779,  77851.83139922])

In [30]:
def rrmse(ref_, pred_, tolerance=1e-3):

    er_ = []
    for i in range(np.shape(ref_)[0]):
        ref = ref_[i, :]
        pred = pred_[i, :]
        maxref = np.max(np.abs(ref))
        if maxref < tolerance:
            denom_field = 1.
        else:
            denom_field = maxref
        reldif = (pred - ref) / denom_field
        er_.append((1 / ref.shape[0]) * (np.linalg.norm(reldif))**2)

    return np.sqrt(np.mean(np.array(er_)))


In [31]:
def agg_metrics(y_true, y_pred):
    rmse_ = []
    r2_ = []
    for i in range(np.shape(y_true)[0]):
        rmse_.append(mean_squared_error(y_true[i, :], y_pred[i, :], squared=False))
        r2_.append(r2_score(y_true[i, :], y_pred[i, :]))
    rmse_ = np.mean(np.array(rmse_))
    r2_ = np.mean(np.array(r2_))

    rrmse_ = rrmse(y_true, y_pred)

    print('RMSE: {:.3e} - RRMSE: {:.3e} - R2: {:.3e}'.format(rmse_, rrmse_, r2_))
    return rmse_, rrmse_, r2_


In [41]:
import modred as mr
num_vecs = 30
vecs = np.random.random((100, num_vecs))
# Compute POD
num_modes = 5
POD_res = mr.compute_POD_arrays_snaps_method(
    vecs, list(mr.range(num_modes)))
modes = POD_res.modes
eigvals = POD_res.eigvals


In [38]:
np.shape(y_test)

(200, 29773)

In [39]:

U, S, Vh = np.linalg.svd(y_test[:3,::1000], full_matrices=False)


In [ ]:
# from UQpy.dimension_reduction import DirectPOD, SnapshotPOD, HigherOrderSVD
# n_modes = [1, 2]
# Data_modes = []
# for n_mode in n_modes:
#     pod = DirectPOD(solution_snapshots=x_train, n_modes=n_mode)
#     Data_reconstr = pod.reconstructed_solution
#     Data_reduced = pod.reduced_solution
#     Data_modes.append(Data_reconstr)
#     del Data_reconstr



In [ ]:
npca_input = 50
npca_output = 40

scaler_input = MinMaxScaler()
scaler_output = MinMaxScaler()
pca_input = PCA(npca_input)
pca_output = PCA(npca_output)

pca_input.fit(scaler_input.fit_transform(np.concatenate((x_train, x_test), axis=0)))

pc_input_train = pca_input.transform(scaler_input.transform(x_train))
pc_input_test = pca_input.transform(scaler_input.transform(x_test))
pc_output_train = pca_output.fit_transform(scaler_output.fit_transform(y_train))
pc_output_test = pca_output.transform(scaler_output.transform(y_test))

print(np.cumsum(pca_input.explained_variance_ratio_))
print(np.cumsum(pca_output.explained_variance_ratio_))

y_train_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_train))
y_test_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_test))

_ = agg_metrics(y_train, y_train_)
_ = agg_metrics(y_test, y_test_)


In [ ]:
# engine = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

# engine = MultiOutputRegressor(XGBRegressor(n_estimators=100,
#                                            max_depth=5,
#                                            eta=0.1,
#                                            subsample=0.7,
#                                            colsample_bytree=0.8))

engine = MultiOutputRegressor(Pipeline([('scaler', MinMaxScaler()),
                                        ('model', GaussianProcessRegressor(kernel = Matern(length_scale=1.0,
                                                                                           length_scale_bounds=(1e-20, 1e10),
                                                                                           nu=1.5), #+ DotProduct() + WhiteKernel(), # + RBF(),
                                                                           alpha=1e-10,
                                                                           normalize_y=True))]))

# engine = MultiOutputRegressor(Pipeline([('poly', PolynomialFeatures(degree=2)),
#                                         ('scaler', MinMaxScaler()),
#                                         ('model', Ridge(alpha=0.001))]))

# engine = MultiOutputRegressor(Pipeline([('poly', PolynomialFeatures(degree=2)),
#                                         ('scaler', MinMaxScaler()),
#                                         ('model', Ridge(alpha=0.001))]))

# engine = MultiOutputRegressor(Pipeline([('poly', PolynomialFeatures(degree=2)),
#                                         ('scaler', MinMaxScaler()),
#                                         ('model', BayesianRidge(tol=1e-10, fit_intercept=False))]))

# engine = MultiOutputRegressor(Pipeline([('poly', PolynomialFeatures(degree=2)),
#                                         ('scaler', MinMaxScaler()),
#                                         ('model', MLPRegressor([32,32,32], max_iter=500))]))

x_train_in = np.concatenate((pc_input_train, x_scalar_train), axis=1)
x_test_in = np.concatenate((pc_input_test, x_scalar_test), axis=1)

engine.fit(x_train_in, pc_output_train)

pc_output_train_ = engine.predict(x_train_in)
pc_output_test_ = engine.predict(x_test_in)

y_train_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_train_))
y_test_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_test_))

print('TRAIN')
_ = agg_metrics(y_train, y_train_)
print('TEST')
_ = agg_metrics(y_test, y_test_)


In [ ]:
print(r2_score(pc_output_train, pc_output_train_))
print(r2_score(pc_output_test, pc_output_test_))


In [ ]:
print(r2_score(y_train.T, y_train_.T))
print(r2_score(y_test.T, y_test_.T))


In [ ]:
engine = MultiOutputRegressor(Pipeline([
                                        ('scaler', MinMaxScaler()),
                                        ('poly', PolynomialFeatures(degree=2)),
                                        ('model', BayesianRidge(tol=1e-10, fit_intercept=True))]))
engine.fit(x_train_in, pc_output_train)

pc_output_train_ = engine.predict(x_train_in)
pc_output_test_ = engine.predict(x_test_in)

y_train_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_train_))
y_test_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_test_))

print('TRAIN')
_ = agg_metrics(y_train, y_train_)
print('TEST')
_ = agg_metrics(y_test, y_test_)


In [ ]:
engine = MultiOutputRegressor(Pipeline([('scaler', MinMaxScaler()),
                                        ('model', MLPRegressor([128,128,128], max_iter=500))]))
engine.fit(x_train_in, pc_output_train)

pc_output_train_ = engine.predict(x_train_in)
pc_output_test_ = engine.predict(x_test_in)

y_train_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_train_))
y_test_ = scaler_output.inverse_transform(pca_output.inverse_transform(pc_output_test_))

print('TRAIN')
_ = agg_metrics(y_train, y_train_)
print('TEST')
_ = agg_metrics(y_test, y_test_)
